In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: QuikGraph, 2.3.0"
#r "/Users/jnmatthews/MGGG/GerryChain.NET/GerryChain/bin/Release/net5.0/GerryChain.dll"

Installed Packages Newtonsoft.Json, 13.0.1 Plotly.NET, 2.0.0-preview.14 Plotly.NET.Interactive, 2.0.0-preview.14 QuikGraph, 2.3.0

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.14"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.14"

# Running a County Aware chain in AL and plotting observed BVAP shares

In [ ]:
using System;
using System.Linq;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using GerryChain;
using Plotly.NET;
using Plotly.NET.Interactive;

In [ ]:
var TallyBVAP = Scores.TallyFactory("BVAP", "BVAP");
var TallyVAP = Scores.TallyFactory("VAP", "VAP");
var TallyPOP = Scores.TallyFactory("TOTPOP", "TOTPOP");
var initPart = new Partition("../resources/al_vtds20_with_seeds.json", "CD_Seed", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" },
                             new Score[] { TallyBVAP, TallyPOP, TallyVAP }, regionAware: true, regionDivisionSpecs: new (string, double)[]{("COUNTY", 1)});
var chain = new Chain(initPart, 1000, 0.01, batchSize:4);

In [ ]:
var bs = chain.Select(p => ((DistrictWideScoreValue) p.Score("BVAP")).Value.Zip(((DistrictWideScoreValue)p.Score("VAP")).Value, (b, v) => b/v)).ToArray();

In [ ]:
var bvaps = bs.Select(v => v.OrderBy(v => v).ToArray()).ToArray();

In [ ]:
double[] xs = bs.SelectMany(v => v.OrderBy(v => v).ToArray()).ToArray();

In [ ]:
int[] bins = Enumerable.Range(0, 100).SelectMany(_ => Enumerable.Range(1,7)).ToArray();
string[] binsStr = bins.Select(b => b.ToString()).ToArray();

In [ ]:
GenericChart.GenericChart chart = Chart2D.Chart.BoxPlot<string, double, int>(binsStr, xs)
    .WithXAxisStyle(title: Title.init("District"), ShowGrid: false, ShowLine: true)
    .WithYAxisStyle(title: Title.init("% BVAP share"), ShowGrid: false, ShowLine: true);
    // .Show();

In [ ]:
chart

<!-- Plotly chart will be drawn inside this DIV -->

## Save last partition to file.

In [ ]:
using System.IO;

In [ ]:
var p = chain.Last();

In [ ]:
JObject rss = new JObject(new JProperty("assignments",
                                        new JObject(p.Assignments.Select((n,i) => new JProperty(i.ToString(), n)))));


In [ ]:
File.WriteAllText("last_assignment.json", rss.ToString());

## Optimization Run

In [ ]:
var TallyPOP = Scores.TallyFactory("TOTPOP", "TOTPOP");
var scores = Scores.MinShareOverThresholdPlusNextHighest("GinglesBVAP","BVAP", "VAP", 0.5);
scores.Append(TallyPOP);
var initPart = new Partition("../resources/al_vtds20_with_seeds.json", "CD_Seed", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" },
                             scores, regionAware: true, regionDivisionSpecs: new (string, double)[]{("COUNTY", 1)});
var accept = AcceptanceFunctions.SimulatedAnnealingFactory(initPart, "GinglesBVAP", 133, 100, 100);

var chainOpt = new Chain(initPart, 1000, 0.01, batchSize:4, accept: accept);

In [ ]:
var bs = chainOpt.Select(p => ((PlanWideScoreValue)p.Score("GinglesBVAP")).Value).ToArray();

In [ ]:
GenericChart.GenericChart chart = Chart2D.Chart.Line<int, double, int>(Enumerable.Range(0, 1000), bs)
    .WithXAxisStyle(title: Title.init("Step"), ShowGrid: false, ShowLine: true)
    .WithYAxisStyle(title: Title.init("BVAP Gingles"), ShowGrid: false, ShowLine: true);
  

In [ ]:
chart

<!-- Plotly chart will be drawn inside this DIV -->